not good


random forest for refined dataset that selects features with small p-value
- imputed with mean value of each column
- eliminate outliers and leverges after normalized
- select features with small p-value

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import normalize
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

import scipy

In [3]:
plt.rcParams["figure.figsize"] = (15, 10)

## load data

In [4]:
pth_data = './processed_data/train_selected_not_norm.csv'
df_selected = pd.read_csv(pth_data)

In [5]:
df_selected

,height,weight,gender,bmi,rh,stress_level,skin_temp,TC,TS,Clo,env_temp
0,1.60,52.5,1.0,20.507812,62.985,19.500000,32.003449,4.0,4.0,0.61,75.942
1,1.77,58.0,-1.0,18.513199,48.702,2.000000,30.933000,4.0,4.0,0.36,77.378
2,1.71,64.0,-1.0,21.887076,47.990,33.000000,33.199000,4.0,4.0,0.61,75.484
3,1.77,58.0,-1.0,18.513199,52.445,53.000000,30.431000,4.0,4.0,0.36,77.203
4,1.77,58.0,-1.0,18.513199,56.396,80.000000,30.087000,4.0,4.0,0.36,77.902
...,...,...,...,...,...,...,...,...,...,...,...
2270,1.77,90.0,-1.0,28.727377,59.397,33.000000,31.353000,4.0,4.0,0.36,74.467
2271,1.71,64.0,-1.0,21.887076,47.190,32.111013,32.003449,4.0,4.0,0.74,73.753
2272,1.66,57.0,1.0,20.685150,52.940,2.000000,30.854000,4.0,4.0,0.54,76.703
2273,1.69,75.0,-1.0,26.259585,47.010,32.111013,31.300000,3.0,4.0,0.36,75.484


In [6]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2275 entries, 0 to 2274
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   height        2275 non-null   float64
 1   weight        2275 non-null   float64
 2   gender        2275 non-null   float64
 3   bmi           2275 non-null   float64
 4   rh            2275 non-null   float64
 5   stress_level  2275 non-null   float64
 6   skin_temp     2275 non-null   float64
 7   TC            2275 non-null   float64
 8   TS            2275 non-null   float64
 9   Clo           2275 non-null   float64
 10  env_temp      2275 non-null   float64
dtypes: float64(11)
memory usage: 195.6 KB


## data process

### X & y

In [7]:
X_data = df_selected.iloc[:, :-1]

In [8]:
X_cols_name = X_data.columns

In [9]:
y_data = df_selected['env_temp']

### normalize

In [10]:
X_array_normalize, X_norm = normalize(X_data, axis=0, return_norm=True)

In [11]:
X_data = pd.DataFrame(X_array_normalize, columns=X_cols_name)

In [12]:
X_data

,height,weight,gender,bmi,rh,stress_level,skin_temp,TC,TS,Clo
0,0.019684,0.017241,0.020966,0.019628,0.025350,0.010726,0.020960,0.021498,0.018985,0.023903
1,0.021776,0.019047,-0.020966,0.017719,0.019601,0.001100,0.020259,0.021498,0.018985,0.014107
2,0.021038,0.021017,-0.020966,0.020949,0.019315,0.018151,0.021743,0.021498,0.018985,0.023903
3,0.021776,0.019047,-0.020966,0.017719,0.021108,0.029152,0.019930,0.021498,0.018985,0.014107
4,0.021776,0.019047,-0.020966,0.017719,0.022698,0.044003,0.019705,0.021498,0.018985,0.014107
...,...,...,...,...,...,...,...,...,...,...
2270,0.021776,0.029556,-0.020966,0.027496,0.023906,0.018151,0.020534,0.021498,0.018985,0.014107
2271,0.021038,0.021017,-0.020966,0.020949,0.018993,0.017662,0.020960,0.021498,0.018985,0.028997
2272,0.020423,0.018719,0.020966,0.019798,0.021307,0.001100,0.020207,0.021498,0.018985,0.021160
2273,0.020792,0.024630,-0.020966,0.025134,0.018920,0.017662,0.020499,0.016124,0.018985,0.014107


In [13]:
X_norm

array([  81.28230804, 3045.10792091,   47.69696007, 1044.80010521,
       2484.63715978, 1818.04462895, 1526.87286098,  186.06181768,
        210.69409104,   25.51961207])

## random forest (1st stry)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=1208)

In [15]:
rfr = RandomForestRegressor()

In [16]:
rfr.fit(X_train, y_train)

RandomForestRegressor()

In [17]:
y_pred = rfr.predict(X_test)

In [18]:
# mean squared error
mean_squared_error(y_test, y_pred)

1.496873368793176

In [19]:
# R2 score
r2_score(y_test, y_pred)

0.6879814879755035